Image Preprocessing

In [10]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='tGcNDmAc_N8W25Ld_PCWuPUl9MOPW3PLmzlOU0XjZCI2',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'traincnnmodel-donotdelete-pr-y2q0cidugru0d7'
object_key = 'conversation_engine_for_deaf_and_dumb.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
from io import BytesIO
import zipfile
zip_ref = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths=zip_ref.namelist()
for path in file_paths:
    zip_ref.extract(path)

In [14]:
f=os.listdir('/home/wsuser/work/Dataset/training_set')
f

['B', 'E', 'F', 'G', 'D', 'I', 'C', 'H', 'A']

In [15]:
#image Preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale = 1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True) 
test_datagen=ImageDataGenerator (rescale = 1./255)
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Dataset/training_set', target_size=(64,64), batch_size=300, class_mode='categorical', color_mode = "grayscale")
x_test = train_datagen.flow_from_directory('/home/wsuser/work/Dataset/test_set', target_size=(64,64), batch_size=300, class_mode='categorical', color_mode = "grayscale")

Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


# Model Building

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten
model=Sequential()
model.add(Convolution2D(32,(3,3),input_shape=(64,64,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=512,activation='relu'))
model.add(Dense(units=9,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit_generator(x_train, steps_per_epoch=24, epochs=10, validation_data=x_test, validation_steps=40) 

/tmp/wsuser/ipykernel_210/3767310412.py:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train, steps_per_epoch=24, epochs=10, validation_data=x_test, validation_steps=40)


Epoch 1/10
24/24 [==============================] - 23s 950ms/step - loss: 1.2163 - accuracy: 0.6738 - val_loss: 0.5050 - val_accuracy: 0.8818
Epoch 2/10
24/24 [==============================] - 20s 846ms/step - loss: 0.2580 - accuracy: 0.9283
Epoch 3/10
24/24 [==============================] - 21s 853ms/step - loss: 0.1422 - accuracy: 0.9651
Epoch 4/10
24/24 [==============================] - 21s 851ms/step - loss: 0.0847 - accuracy: 0.9803
Epoch 5/10
24/24 [==============================] - 21s 850ms/step - loss: 0.0531 - accuracy: 0.9872
Epoch 6/10
24/24 [==============================] - 20s 826ms/step - loss: 0.0505 - accuracy: 0.9872
Epoch 7/10
24/24 [==============================] - 20s 841ms/step - loss: 0.0361 - accuracy: 0.9926
Epoch 8/10
24/24 [==============================] - 21s 863ms/step - loss: 0.0260 - accuracy: 0.9946
Epoch 9/10
24/24 [==============================] - 20s 833ms/step - loss: 0.0213 - accuracy: 0.9955
Epoch 10/10
24/24 [==============================

In [17]:
model.save('cbot.h5')

In [19]:
!tar -zcvf cbotModel.tgz cbot.h5

cbot.h5


In [21]:
ls

cbot.h5  cbotModel.tgz  Dataset/


In [22]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 24.4 MB/s eta 0:00:01


In [24]:
#ibm watson machine learing Credentials
from ibm_watson_machine_learning import APIClient
wmlC={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"4uZlU1UcvB87fqoarYOEb-x5_awZ3DlCBz7JfwC6xJBZ"
}
client=APIClient(wmlC)

In [28]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [29]:
space_uid=guid_from_space_name(client,'CNNDep')
print("SpaceUID: "+space_uid)

SpaceUID: e72d505f-489e-476c-8d00-5342a7d6b46f


In [30]:
client.set.default_space(space_uid)

'SUCCESS'

In [31]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [40]:
software_spec_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [49]:
model_details=client.repository.store_model(model='cbotModel.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
    
})
model_id=client.repository.get_model_id(model_details)
model_id

'5ca9eb20-97a7-4bf5-a583-c152ee8ef9ad'

In [50]:
client.repository.download(model_id,"cbotModel.tar.gz")

Successfully saved model content to file: 'cbotModel.tar.gz'


'/home/wsuser/work/cbotModel.tar.gz'

# Test The Model

In [63]:
from keras.models import load_model
import numpy as np
import cv2
from keras.preprocessing import image

In [52]:
model=load_model('cbot.h5')

In [57]:
from skimage.transform import resize
def detect(frame):
    img=image.img_to_array(frame)
    img = resize(img,(64,64,1))
    img = np.expand_dims(img,axis=0)
    pred=np.argmax(model.predict(img))
    op=['A','B','C','D','E','F','G','H','I']
    print("THE PREDICTED LETTER IS ",op[pred])

In [65]:
img=image.load_img("/home/wsuser/work/Dataset/test_set/D/102.png")
detect(img)

THE PREDICTED LETTER IS  D
